In [3]:
!pip install pandas scikit-learn

In [4]:
import pandas as pd

small_df = pd.read_csv('/content/small_dataset.csv')
display(small_df.head())

,email_id,customer_id,subject,body,tag
0,1,CUST_A,Unable to access shared mailbox,"Hi team, I'm unable to access the shared mailb...",access_issue
1,2,CUST_A,Rules not working,We created a rule to auto-assign emails based ...,workflow_issue
2,3,CUST_A,Email stuck in pending,One of our emails is stuck in pending even aft...,status_bug
3,4,CUST_B,Automation creating duplicate tasks,Your automation engine is creating 2 tasks for...,automation_bug
4,5,CUST_B,Tags missing,Many of our tags are not appearing for new ema...,tagging_issue


In [6]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from collections import defaultdict

In [10]:
def train_customer_models(df: pd.DataFrame):
    """
    Train one text classifier per customer_id using TF-IDF + LogisticRegression.
    Returns:
      - models: dict[customer_id] -> sklearn Pipeline
      - customer_tags: dict[customer_id] -> set of tags
    """
    models = {}
    customer_tags = {}

    for cust_id, group in df.groupby("customer_id"):
        X = group["text"].tolist()
        y = group["tag"].tolist()

        # For tiny datasets, use a simple solver & balanced class weights
        pipeline = Pipeline([
            ("tfidf", TfidfVectorizer(ngram_range=(1, 2), stop_words="english")),
            ("clf", LogisticRegression(max_iter=1000, class_weight="balanced"))
        ])

        pipeline.fit(X, y)
        models[cust_id] = pipeline
        customer_tags[cust_id] = set(y)

        print(f"Trained model for {cust_id}: {len(group)} examples, {len(set(y))} tags")

    return models, customer_tags

customer_models, customer_tags = train_customer_models(large_df)

Trained model for CUST_A: 10 examples, 10 tags
Trained model for CUST_B: 10 examples, 10 tags
Trained model for CUST_C: 10 examples, 10 tags
Trained model for CUST_D: 10 examples, 9 tags
Trained model for CUST_E: 10 examples, 10 tags
Trained model for CUST_F: 10 examples, 10 tags


In [12]:
def predict_tag_baseline(subject: str, body: str, customer_id: str):
    text = (subject or "") + " " + (body or "")

    if customer_id not in customer_models:
        raise ValueError(f"No model trained for customer_id={customer_id}")

    model = customer_models[customer_id]
    proba = model.predict_proba([text])[0]
    classes = model.classes_
    best_idx = np.argmax(proba)
    return classes[best_idx], float(proba[best_idx])

In [14]:
pred, conf = predict_tag_baseline(
    subject="Unable to access shared mailbox",
    body="I'm getting permissions error",
    customer_id="CUST_A"
)
print(pred, conf)

access_issue 0.18278429136598068


In [16]:
def evaluate_per_customer(df: pd.DataFrame, test_size=0.3, random_state=42):
    reports = {}
    all_y_true = []
    all_y_pred = []

    for cust_id, group in df.groupby("customer_id"):
        X = group["text"].tolist()
        y = group["tag"].tolist()

        if len(group) < 3:
            print(f"Skipping customer {cust_id} (too few examples).")
            continue

        # Check if stratification is possible for this customer's data
        unique_classes, counts = np.unique(y, return_counts=True)
        can_stratify = all(c >= 2 for c in counts)

        if can_stratify:
            X_train, X_test, y_train, y_test = train_test_split(
                X, y, test_size=test_size, random_state=random_state, stratify=y
            )
        else:
            print(f"Warning: Not enough samples for stratified split for customer {cust_id}. Performing non-stratified split.")
            X_train, X_test, y_train, y_test = train_test_split(
                X, y, test_size=test_size, random_state=random_state
            )

        pipe = Pipeline([
            ("tfidf", TfidfVectorizer(ngram_range=(1, 2), stop_words="english")),
            ("clf", LogisticRegression(max_iter=1000, class_weight="balanced"))
        ])
        pipe.fit(X_train, y_train)
        y_pred = pipe.predict(X_test)

        acc = accuracy_score(y_test, y_pred)
        print(f"\n=== {cust_id} ===")
        print("Accuracy:", acc)
        print(classification_report(y_test, y_pred))

        reports[cust_id] = {
            "accuracy": acc,
            "report": classification_report(y_test, y_pred, output_dict=True)
        }

        all_y_true.extend(y_test)
        all_y_pred.extend(y_pred)

    print("\n=== Overall (all customers combined) ===")
    print("Accuracy:", accuracy_score(all_y_true, all_y_pred))
    print(classification_report(all_y_true, all_y_pred))

    return reports

baseline_reports = evaluate_per_customer(large_df)


=== CUST_A ===
Accuracy: 0.0
                  precision    recall  f1-score   support

automation_delay       0.00      0.00      0.00       1.0
 feature_request       0.00      0.00      0.00       0.0
notification_bug       0.00      0.00      0.00       1.0
 threading_issue       0.00      0.00      0.00       0.0
  workflow_issue       0.00      0.00      0.00       1.0

        accuracy                           0.00       3.0
       macro avg       0.00      0.00      0.00       3.0
    weighted avg       0.00      0.00      0.00       3.0


=== CUST_B ===
Accuracy: 0.0
                 precision    recall  f1-score   support

analytics_issue       0.00      0.00      0.00       1.0
feature_request       0.00      0.00      0.00       0.0
     sync_issue       0.00      0.00      0.00       1.0
  tagging_issue       0.00      0.00      0.00       1.0

       accuracy                           0.00       3.0
      macro avg       0.00      0.00      0.00       3.0
   weighted av

/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_


=== CUST_D ===
Accuracy: 0.0
                  precision    recall  f1-score   support

forwarding_issue       0.00      0.00      0.00       0.0
          ui_bug       0.00      0.00      0.00       2.0
    ui_state_bug       0.00      0.00      0.00       1.0

        accuracy                           0.00       3.0
       macro avg       0.00      0.00      0.00       3.0
    weighted avg       0.00      0.00      0.00       3.0


=== CUST_E ===
Accuracy: 0.0
                           precision    recall  f1-score   support

             admin_ui_bug       0.00      0.00      0.00       0.0
          analytics_issue       0.00      0.00      0.00       1.0
         assignment_issue       0.00      0.00      0.00       1.0
         automation_delay       0.00      0.00      0.00       0.0
mobile_notification_issue       0.00      0.00      0.00       1.0

                 accuracy                           0.00       3.0
                macro avg       0.00      0.00      0.00    

/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_

In [18]:
# Simple keyword-to-tag hints (global)
KEYWORD_TO_TAG = {
    "billing": "billing_error",        # or "billing" depending on customer
    "invoice": "billing_error",
    "csat": "analytics_issue",
    "survey": "csat_issue",
    "mail merge": "mail_merge_issue",
    "automation": "automation_issue",
    "sla": "sla_issue",
    "performance": "performance",
    "slow": "performance",
    "lag": "sync_delay",
    "search": "search_issue",
    "feature request": "feature_request",
    "dark mode": "feature_request",
    "api": "information_request",
}

In [21]:
def rule_based_override(text: str, customer_id: str):
    """
    Returns a tag if a strong rule applies, else None.
    Rules are filtered by tags allowed for that customer.
    """
    allowed_tags = customer_tags.get(customer_id, set())
    text_l = text.lower()

    # Strong positive patterns
    for phrase, suggested_tag in KEYWORD_TO_TAG.items():
        if phrase in text_l and suggested_tag in allowed_tags:
            return suggested_tag

    # Anti-pattern: feature requests should not be bugs
    if "feature request" in text_l or "please consider" in text_l:
        for t in allowed_tags:
            if "feature_request" in t:
                return t

    return None

In [23]:
def predict_tag(subject: str, body: str, customer_id: str, use_rules=True):
    text = (subject or "") + " " + (body or "")

    # 1. Customer isolation check
    if customer_id not in customer_models:
        raise ValueError(f"No model trained for customer_id={customer_id}")

    # 2. Rules layer
    if use_rules:
        tag = rule_based_override(text, customer_id)
        if tag is not None:
            return tag, 1.0, "rule_override"

    # 3. Fall back to ML model
    model = customer_models[customer_id]
    proba = model.predict_proba([text])[0]
    classes = model.classes_
    best_idx = np.argmax(proba)
    return classes[best_idx], float(proba[best_idx]), "model"

In [25]:
test_row = small_df.iloc[0]
pred_tag, conf, source = predict_tag(
    subject=test_row["subject"],
    body=test_row["body"],
    customer_id=test_row["customer_id"]
)
print("Predicted:", pred_tag, "| True:", test_row["tag"], "| source:", source, "| conf:", conf)

Predicted: access_issue | True: access_issue | source: model | conf: 0.17890626622990302


In [8]:
def prepare_df(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    df["text"] = df["subject"].fillna("") + " " + df["body"].fillna("")
    return df

small_df = prepare_df(small_df)
large_df = prepare_df(large_df)

print("Columns:", large_df.columns)

Columns: Index(['email_id', 'customer_id', 'subject', 'body', 'tag', 'text'], dtype='object')


In [26]:
def evaluate_with_rules(df: pd.DataFrame):
    y_true = []
    y_pred = []

    errors = []

    for _, row in df.iterrows():
        subject = row["subject"]
        body = row["body"]
        cust = row["customer_id"]
        true_tag = row["tag"]

        try:
            pred_tag, conf, source = predict_tag(subject, body, cust, use_rules=True)
        except ValueError as e:
            print("Skipping:", e)
            continue

        y_true.append(true_tag)
        y_pred.append(pred_tag)

        if pred_tag != true_tag:
            errors.append({
                "customer_id": cust,
                "subject": subject,
                "body": body,
                "true_tag": true_tag,
                "pred_tag": pred_tag,
                "source": source,
                "conf": conf
            })

    print("Accuracy with rules:", accuracy_score(y_true, y_pred))
    print(classification_report(y_true, y_pred))

    error_df = pd.DataFrame(errors)
    print("Some misclassifications:")
    display(error_df.head(10))
    return error_df

error_df = evaluate_with_rules(large_df)

Accuracy with rules: 1.0
                           precision    recall  f1-score   support

             access_issue       1.00      1.00      1.00         1
             admin_ui_bug       1.00      1.00      1.00         1
       analytics_accuracy       1.00      1.00      1.00         1
            analytics_bug       1.00      1.00      1.00         1
          analytics_issue       1.00      1.00      1.00         2
        analytics_latency       1.00      1.00      1.00         1
           assignment_bug       1.00      1.00      1.00         1
         assignment_issue       1.00      1.00      1.00         1
         attachment_issue       1.00      1.00      1.00         1
   attachment_preview_bug       1.00      1.00      1.00         1
               auth_issue       1.00      1.00      1.00         1
           automation_bug       1.00      1.00      1.00         1
         automation_delay       1.00      1.00      1.00         2
         automation_issue       1.00

""


In [7]:
import pandas as pd

small_path = "/content/small_dataset.csv"
large_path = "/content/large_dataset.csv"

small_df = pd.read_csv(small_path)
large_df = pd.read_csv(large_path)

print("Small dataset:")
display(small_df.head())

print("Large dataset:")
display(large_df.head())

Small dataset:


,email_id,customer_id,subject,body,tag
0,1,CUST_A,Unable to access shared mailbox,"Hi team, I'm unable to access the shared mailb...",access_issue
1,2,CUST_A,Rules not working,We created a rule to auto-assign emails based ...,workflow_issue
2,3,CUST_A,Email stuck in pending,One of our emails is stuck in pending even aft...,status_bug
3,4,CUST_B,Automation creating duplicate tasks,Your automation engine is creating 2 tasks for...,automation_bug
4,5,CUST_B,Tags missing,Many of our tags are not appearing for new ema...,tagging_issue


Large dataset:


,email_id,customer_id,subject,body,tag
0,1,CUST_A,Unable to access shared mailbox,I am getting a permission denied message when ...,access_issue
1,2,CUST_A,Rule not triggering,Our auto-assignment rule is no longer firing f...,workflow_issue
2,3,CUST_A,Email threads not merging,Two replies from the same customer created dif...,threading_issue
3,4,CUST_A,Tag suggestions incorrect,Tag suggestions are showing irrelevant tags li...,tagging_accuracy
4,5,CUST_A,Drafts disappearing,Draft replies disappear when switching between...,ui_bug


In [27]:
# 10 sample emails to test sentiment prompt
sentiment_test_emails = [
    "Thanks team, the new update fixed the issue. Great job!",
    "We are still facing the same problem. This is getting frustrating.",
    "Can you please help me configure workflows?",
    "Everything seems fine now, appreciate the quick response!",
    "This is terrible. Nothing works. Very disappointed.",
    "We need dark mode but it's not urgent.",
    "The system is extremely slow and affecting productivity.",
    "Thanks for the feature request update.",
    "Our billing was incorrect AGAIN. Unacceptable.",
    "Quick question: How do I export reports?"
]
sentiment_test_emails

['Thanks team, the new update fixed the issue. Great job!',
 'We are still facing the same problem. This is getting frustrating.',
 'Can you please help me configure workflows?',
 'Everything seems fine now, appreciate the quick response!',
 'This is terrible. Nothing works. Very disappointed.',
 "We need dark mode but it's not urgent.",
 'The system is extremely slow and affecting productivity.',
 'Thanks for the feature request update.',
 'Our billing was incorrect AGAIN. Unacceptable.',
 'Quick question: How do I export reports?']

In [30]:
import re

def sentiment_analysis_v1(text):
    text_lower = text.lower()

    positive_words = ["thanks", "appreciate", "great", "good", "fixed"]
    negative_words = ["frustrating", "terrible", "disappointed", "slow", "incorrect", "unacceptable", "issue"]

    score = 0

    for w in positive_words:
        if w in text_lower:
            score += 1

    for w in negative_words:
        if w in text_lower:
            score -= 1

    if score > 0:
        sentiment = "positive"
        confidence = min(0.4 + (score * 0.2), 1.0)
    elif score < 0:
        sentiment = "negative"
        confidence = min(0.4 + (abs(score) * 0.2), 1.0)
    else:
        sentiment = "neutral"
        confidence = 0.5

    reasoning = f"Detected score={score} based on keyword matches."

    return {"sentiment": sentiment, "confidence": confidence, "reasoning": reasoning}

In [31]:
sentiment_results_v1 = []

for email in sentiment_test_emails:
    result = sentiment_analysis_v1(email)
    sentiment_results_v1.append({"email": email, **result})

import pandas as pd
df_sentiment_v1 = pd.DataFrame(sentiment_results_v1)
df_sentiment_v1

,email,sentiment,confidence,reasoning
0,"Thanks team, the new update fixed the issue. G...",positive,0.8,Detected score=2 based on keyword matches.
1,We are still facing the same problem. This is ...,negative,0.6,Detected score=-1 based on keyword matches.
2,Can you please help me configure workflows?,neutral,0.5,Detected score=0 based on keyword matches.
3,"Everything seems fine now, appreciate the quic...",positive,0.6,Detected score=1 based on keyword matches.
4,This is terrible. Nothing works. Very disappoi...,negative,0.8,Detected score=-2 based on keyword matches.
5,We need dark mode but it's not urgent.,neutral,0.5,Detected score=0 based on keyword matches.
6,The system is extremely slow and affecting pro...,negative,0.6,Detected score=-1 based on keyword matches.
7,Thanks for the feature request update.,positive,0.6,Detected score=1 based on keyword matches.
8,Our billing was incorrect AGAIN. Unacceptable.,negative,0.8,Detected score=-2 based on keyword matches.
9,Quick question: How do I export reports?,neutral,0.5,Detected score=0 based on keyword matches.


In [32]:
df_sentiment_v1["human_label"] = [
    "positive", "negative", "neutral", "positive", "negative",
    "neutral", "negative", "positive", "negative", "neutral"
]

df_sentiment_v1["correct?"] = (df_sentiment_v1["human_label"] == df_sentiment_v1["sentiment"])
df_sentiment_v1

,email,sentiment,confidence,reasoning,human_label,correct?
0,"Thanks team, the new update fixed the issue. G...",positive,0.8,Detected score=2 based on keyword matches.,positive,True
1,We are still facing the same problem. This is ...,negative,0.6,Detected score=-1 based on keyword matches.,negative,True
2,Can you please help me configure workflows?,neutral,0.5,Detected score=0 based on keyword matches.,neutral,True
3,"Everything seems fine now, appreciate the quic...",positive,0.6,Detected score=1 based on keyword matches.,positive,True
4,This is terrible. Nothing works. Very disappoi...,negative,0.8,Detected score=-2 based on keyword matches.,negative,True
5,We need dark mode but it's not urgent.,neutral,0.5,Detected score=0 based on keyword matches.,neutral,True
6,The system is extremely slow and affecting pro...,negative,0.6,Detected score=-1 based on keyword matches.,negative,True
7,Thanks for the feature request update.,positive,0.6,Detected score=1 based on keyword matches.,positive,True
8,Our billing was incorrect AGAIN. Unacceptable.,negative,0.8,Detected score=-2 based on keyword matches.,negative,True
9,Quick question: How do I export reports?,neutral,0.5,Detected score=0 based on keyword matches.,neutral,True


In [33]:
def sentiment_analysis_v2(text):
    lower = text.lower()

    # Tone-based rules
    strong_negative = ["unacceptable", "terrible", "very disappointed", "angry"]
    soft_negative = ["slow", "issue", "problem", "incorrect"]
    strong_positive = ["amazing", "great job", "excellent", "love it"]
    soft_positive = ["thanks", "appreciate", "helpful"]

    score = 0

    for w in strong_positive:
        if w in lower:
            score += 3

    for w in soft_positive:
        if w in lower:
            score += 1

    for w in strong_negative:
        if w in lower:
            score -= 3

    for w in soft_negative:
        if w in lower:
            score -= 1

    # Classification
    if score >= 2:
        sentiment = "positive"
    elif score <= -2:
        sentiment = "negative"
    else:
        sentiment = "neutral"

    # Confidence calibration
    confidence = min(0.5 + abs(score) * 0.2, 1.0)

    reasoning = f"Score={score}; phrases detected matched sentiment rules."

    return {"sentiment": sentiment, "confidence": confidence, "reasoning": reasoning}

df_sentiment_v2 = pd.DataFrame([
    {"email": email, **sentiment_analysis_v2(email)} for email in sentiment_test_emails
])
df_sentiment_v2["human_label"] = df_sentiment_v1["human_label"]
df_sentiment_v2["correct?"] = (df_sentiment_v2["human_label"] == df_sentiment_v2["sentiment"])
df_sentiment_v2

,email,sentiment,confidence,reasoning,human_label,correct?
0,"Thanks team, the new update fixed the issue. G...",positive,1.0,Score=3; phrases detected matched sentiment ru...,positive,True
1,We are still facing the same problem. This is ...,neutral,0.7,Score=-1; phrases detected matched sentiment r...,negative,False
2,Can you please help me configure workflows?,neutral,0.5,Score=0; phrases detected matched sentiment ru...,neutral,True
3,"Everything seems fine now, appreciate the quic...",neutral,0.7,Score=1; phrases detected matched sentiment ru...,positive,False
4,This is terrible. Nothing works. Very disappoi...,negative,1.0,Score=-6; phrases detected matched sentiment r...,negative,True
5,We need dark mode but it's not urgent.,neutral,0.5,Score=0; phrases detected matched sentiment ru...,neutral,True
6,The system is extremely slow and affecting pro...,neutral,0.7,Score=-1; phrases detected matched sentiment r...,negative,False
7,Thanks for the feature request update.,neutral,0.7,Score=1; phrases detected matched sentiment ru...,positive,False
8,Our billing was incorrect AGAIN. Unacceptable.,negative,1.0,Score=-4; phrases detected matched sentiment r...,negative,True
9,Quick question: How do I export reports?,neutral,0.5,Score=0; phrases detected matched sentiment ru...,neutral,True


In [35]:
import os
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd

# Load KB articles into a dataframe
def load_kb_articles(kb_dir="/content/kb_articles"):
    docs = []
    for fname in os.listdir(kb_dir):
        full_path = os.path.join(kb_dir, fname)
        if os.path.isfile(full_path) and fname.lower().endswith((".txt", ".md")):
            with open(full_path, "r", encoding="utf-8", errors="ignore") as f:
                content = f.read()
            docs.append({"id": fname, "content": content})
    return pd.DataFrame(docs)

kb_df = load_kb_articles()
print(f"Loaded {len(kb_df)} KB articles.")
display(kb_df.head())

Loaded 10 KB articles.


,id,content
0,performance_issues.txt,Title: Fixing Email Loading or UI Performance ...
1,automation_setup.txt,Title: How to Configure Automations in Hiver\n...
2,csat_missing.txt,Title: Why is CSAT Not Appearing?\n\nIf CSAT s...
3,mail_merge_troubleshooting.txt,Title: Troubleshooting Mail Merge Issues\n\nIf...
4,notifications_delay.txt,Title: Why Notifications May Be Delayed\n\nCom...


In [36]:
vectorizer = TfidfVectorizer(stop_words="english")
kb_tfidf_matrix = vectorizer.fit_transform(kb_df["content"])

In [38]:
def retrieve_articles(query: str, top_k: int = 3):
    query_vec = vectorizer.transform([query])
    similarity_scores = cosine_similarity(query_vec, kb_tfidf_matrix)[0]

    results = kb_df.copy()
    results["similarity"] = similarity_scores
    results = results.sort_values(by="similarity", ascending=False).head(top_k)
    return results

In [40]:
def generate_answer(query: str, top_k: int = 3):
    retrieved = retrieve_articles(query, top_k)

    if retrieved.iloc[0]["similarity"] == 0:
        return retrieved, "No relevant information found.", 0.0

    best_doc = retrieved.iloc[0]
    text = best_doc["content"]

    # Simple extractive snippet (first 350 characters)
    snippet = text[:350].replace("\n", " ").strip()

    confidence = float(best_doc["similarity"])

    final_answer = (
        f"📄 Source Article: {best_doc['id']}\n"
        f"💡 Summary:\n{snippet} ..."
    )

    return retrieved, final_answer, confidence

In [45]:
queries = [
    "How do I configure automations in Hiver?",
    "Why is CSAT not appearing?"
]

rag_output = []

for q in queries:
    retrieved_docs, answer, confidence = generate_answer(q)

    print("\n" + "="*70)
    print(f" QUERY: {q}")
    print("="*70)

    print("\n Retrieved Articles:")
    display(retrieved_docs[["id", "similarity"]])

    print("\n Generated Answer:")
    print(answer)

    print("\nConfidence Score:", round(confidence, 3))

    rag_output.append({
        "query": q,
        "retrieved_articles": list(retrieved_docs["id"]),
        "answer": answer,
        "confidence": confidence
    })


 QUERY: How do I configure automations in Hiver?

 Retrieved Articles:


,id,similarity
1,automation_setup.txt,0.371992
4,notifications_delay.txt,0.049169
0,performance_issues.txt,0.032112



 Generated Answer:
📄 Source Article: automation_setup.txt
💡 Summary:
Title: How to Configure Automations in Hiver  Automations in Hiver allow teams to automatically assign conversations, update statuses, and trigger workflows based on conditions.  Steps: 1. Go to Hiver Settings > Automations. 2. Click "Create Automation". 3. Choose a trigger such as:    - New Email Received    - Tag Applied    - Assignee Updated 4. ...

Confidence Score: 0.372

 QUERY: Why is CSAT not appearing?

 Retrieved Articles:


,id,similarity
2,csat_missing.txt,0.545439
0,performance_issues.txt,0.000000
1,automation_setup.txt,0.000000



 Generated Answer:
📄 Source Article: csat_missing.txt
💡 Summary:
Title: Why is CSAT Not Appearing?  If CSAT scores or survey responses are not visible in Analytics, try the following:  Common Causes: - CSAT feature is disabled for the shared inbox. - Surveys are not configured for the correct workflow. - Email volume is too low to generate reports. - Time filters may hide available results.  Fix Steps: 1. Go to ...

Confidence Score: 0.545


In [44]:
failure_cases = [item for item in rag_output if item["confidence"] < 0.20]
print("\n Failure Cases Detected:")
failure_cases


 Failure Cases Detected:


[]